In [1]:
import inspect
from collections import namedtuple
from abc import ABC, abstractmethod

# Classic Strategy

In [2]:
Customer = namedtuple('Customer', 'name fidelity')

In [3]:
class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity

In [4]:
class Order:  # context
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [5]:
class Promotion(ABC):  # the Strategy: an abstract base class
    
    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""

In [6]:
class FidelityPromo(Promotion):  # First Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

In [7]:
class BulkItemPromo(Promotion): # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

In [8]:
class LargeOrderPromo(Promotion): # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [9]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [
    LineItem('banana', 4, .5),
    LineItem('apple', 10, 1.5),
    LineItem('watermellon', 5, 5.0)
]

In [10]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [11]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [12]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [13]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [14]:
long_order = [LineItem(str(item_code), 1, 1.0)
    for item_code in range(10)]

In [15]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [16]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

## Function-Oriented Strategy
Each concrete strategy in Example is a class with a single method, discount. Furthermore, the strategy instances have no state (no instance attributes). You could say they look a lot like plain functions, and you would be right.

In [17]:
Customer = namedtuple('Customer', 'name fidelity')

In [18]:
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    def total(self):
        return self.price * self.quantity

In [19]:
class Order: # the Context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [20]:
def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

In [21]:
def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

In [22]:
def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [23]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
    LineItem('apple', 10, 1.5),
    LineItem('watermellon', 5, 5.0)]

In [24]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [25]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [26]:
banana_cart = [LineItem('banana', 30, .5),
    LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [27]:
long_order = [LineItem(str(item_code), 1, 1.0)
    for item_code in range(10)]
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [28]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00>

### best_promo finds the maximum discount iterating over a list of functions

In [29]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

In [30]:
def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

### Finding Strategies in a Module
Modules in Python are also first-class objects, and the standard library provides several functions to handle them.

In [31]:
promos = [globals()[name] for name in globals()
    if name.endswith('_promo')
    and name != 'best_promo']

In [32]:
def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

### The promos list is built by introspection of a new promotions module
Another way of collecting the available promotions would be to create a module and put all the strategy functions there, except for best_promo.

In [33]:
# promos = [func for name, func in
#           inspect.getmembers(promotions, inspect.isfunction)]

In [34]:
# def best_promo(order):
#     """Select best discount available
#     """
#     return max(promo(order) for promo in promos)